# **Exercise:** *Data Processing*
### July 18, 2022

In [41]:
# Imports
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from config import ml_songs_data
from datetime import datetime
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None

# Get raw data
raw_song_data = pd.read_csv(ml_songs_data)

# Check data types
raw_song_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Index                   1994 non-null   int64  
 1   Title                   1991 non-null   object 
 2   Artist                  1986 non-null   object 
 3   Top Genre               1986 non-null   object 
 4   Year                    1987 non-null   float64
 5   Month                   1994 non-null   object 
 6   Beats Per Minute (BPM)  1985 non-null   float64
 7   Energy                  1986 non-null   float64
 8   Danceability            970 non-null    float64
 9   Loudness (dB)           1987 non-null   float64
 10  Liveness                1986 non-null   float64
 11  Valence                 960 non-null    float64
 12  Length (Duration)       1985 non-null   object 
 13  Acousticness            1986 non-null   float64
 14  Speechiness             1990 non-null   

In [42]:
# Remove rows and columns with less than 50% values
essential_song_data = raw_song_data.copy()

# Remove columns with less than 50% of values
remove_cols = []
for i in range(len(essential_song_data.columns)):
    if (essential_song_data[essential_song_data.columns[i]].isna().sum() >= len(essential_song_data)/2):
        remove_cols.append(essential_song_data.columns[i])
essential_song_data = essential_song_data.drop(columns=remove_cols)

# Remove rows with less than 50% of values
remove_rows = []
for i in range(len(essential_song_data)):
    if (essential_song_data.iloc[i].isna().sum() >= len(essential_song_data.columns)/2):
        remove_rows.append(i)
essential_song_data = essential_song_data.drop(index=remove_rows)

In [43]:
# Fix data hygiene typing issues
hygienic_song_data = essential_song_data.copy()

# Remove Title, Artist, and Index because it will not be easily passed in ML algorithm
hygienic_song_data = hygienic_song_data.drop(columns=['Title', 'Artist', 'Index'])

# Convert all month variables into ints
month_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
hygienic_song_data['Month'] = hygienic_song_data['Month'].replace(month_dict)
hygienic_song_data['Month'] = hygienic_song_data['Month'].astype(int)

# Remove commas from Length Duration and cast as integer
hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()] = hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()].str.replace(',', '')
hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()] = hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()].astype(int)

# Pass all numeric data through mean imputation
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
numeric_data = hygienic_song_data.copy()
numeric_data = numeric_data.drop(columns='Top Genre')
imputer_mean.fit(numeric_data)
numeric_data[:] = imputer_mean.transform(numeric_data)

# Add age 
numeric_data['Year'][numeric_data['Year'] == 92] = 1992
numeric_data['Age'] = 0
for i in range(len(numeric_data)):
    numeric_data['Age'].iloc[i] = (datetime.now() - pd.to_datetime(f'{int(numeric_data["Year"].iloc[i])}-{int(numeric_data["Month"].iloc[i])}-1', yearfirst=True)) / timedelta(days=365)
numeric_data = numeric_data.drop(columns=['Year', 'Month'])

# NEW: convert to standard z-scores
ss = StandardScaler()
ss.fit(numeric_data)
numeric_data[:] = ss.transform(numeric_data)

# Pass all categorical data through mode imputation
imputer_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_data = pd.DataFrame(hygienic_song_data['Top Genre'].copy())
imputer_mode.fit(categorical_data)
categorical_data[:] = imputer_mode.transform(categorical_data)
categorical_data = pd.get_dummies(categorical_data, drop_first=True)

# Put data back together
hygienic_song_data = pd.concat([numeric_data, categorical_data], axis=1)

In [44]:
# View characteristics of numerical data
hygienic_song_data.describe()

,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,Popularity,Age,Top Genre_acoustic pop,...,Top Genre_reggae,Top Genre_reggae fusion,Top Genre_rock-and-roll,Top Genre_scottish singer-songwriter,Top Genre_soft rock,Top Genre_stomp and holler,Top Genre_streektaal,Top Genre_trance,Top Genre_uk pop,Top Genre_yacht rock
count,1.986000e+03,1.986000e+03,1.986000e+03,1.986000e+03,1.986000e+03,1.986000e+03,1.986000e+03,1.986000e+03,1.986000e+03,1986.000000,...,1986.000000,1986.000000,1986.000000,1986.000000,1986.000000,1986.000000,1986.000000,1986.000000,1986.000000,1986.000000
mean,7.871068e-17,-2.826429e-16,-2.289765e-16,-4.829973e-17,2.325543e-16,-6.439964e-17,8.586619e-17,4.293310e-17,-5.724413e-17,0.002014,...,0.004028,0.002014,0.000504,0.001007,0.000504,0.000504,0.000504,0.000504,0.000504,0.000504
std,1.000252e+00,1.000252e+00,1.000252e+00,1.000252e+00,1.000252e+00,1.000252e+00,1.000252e+00,1.000252e+00,1.000252e+00,0.044845,...,0.063356,0.044845,0.022439,0.031726,0.022439,0.022439,0.022439,0.022439,0.022439,0.022439
min,-3.678105e+00,-2.557969e+00,-4.930159e+00,-1.017099e+00,-1.810179e+00,-9.968961e-01,-6.790781e-01,-3.378387e+00,-1.641680e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.421776e-01,-7.967893e-01,-5.438611e-01,-5.987777e-01,-5.401891e-01,-8.934723e-01,-4.521089e-01,-7.320493e-01,-8.971112e-01,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-4.314740e-02,6.122138e-02,4.426133e-03,-4.194970e-01,-1.826710e-01,-3.763534e-01,-2.251397e-01,1.732767e-01,-2.094323e-02,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.510283e-01,8.289151e-01,8.268570e-01,2.378656e-01,2.922412e-01,7.268336e-01,1.829475e-03,8.000408e-01,8.670110e-01,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,6.457834e+00,1.822401e+00,1.923432e+00,4.779644e+00,1.226643e+01,2.416089e+00,1.135029e+01,2.819614e+00,2.281192e+00,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
# Find outliers
def findOutliers(column):
    
    # Get mean and SD
    mean = column.mean()
    std = column.std()

    # Get list of outliers (i.e., 3 STD from total)
    outliers = []
    for i in range(len(column)):
        val = column.iloc[i]
        if (val < mean - 3 * std or val > mean + 3 * std):
            outliers.append(i)
    
    # Return outlier indices
    return outliers

In [46]:
# Get mean, std, and outliers for each column
for column in hygienic_song_data.columns:
    print(f'  Column: {column}')
    print(f'    Mean: {hygienic_song_data[column].mean()}')
    print(f'     STD: {hygienic_song_data[column].std()}')
    print(f'Outliers: {len(findOutliers(hygienic_song_data[column]))}')
    print()

  Column: Beats Per Minute (BPM)
    Mean: 7.87106756632538e-17
     STD: 1.0002518574526775
Outliers: 4

  Column: Energy
    Mean: -2.82642880790775e-16
     STD: 1.0002518574526775
Outliers: 0

  Column: Loudness (dB)
    Mean: -2.289765110203747e-16
     STD: 1.0002518574526775
Outliers: 22

  Column: Liveness
    Mean: -4.829973279336028e-17
     STD: 1.0002518574526775
Outliers: 57

  Column: Length (Duration)
    Mean: 2.3255426900506804e-16
     STD: 1.0002518574526775
Outliers: 26

  Column: Acousticness
    Mean: -6.439964372448039e-17
     STD: 1.0002518574526775
Outliers: 0

  Column: Speechiness
    Mean: 8.58661916326405e-17
     STD: 1.0002518574526773
Outliers: 42

  Column: Popularity
    Mean: 4.293309581632025e-17
     STD: 1.0002518574526775
Outliers: 9

  Column: Age
    Mean: -5.724412775509367e-17
     STD: 1.0002518574526775
Outliers: 0

  Column: Top Genre_acoustic pop
    Mean: 0.002014098690835851
     STD: 0.04484478467966559
Outliers: 4

  Column: Top Genre

In [47]:
# Save data
hygienic_song_data.to_csv('data/song_data_clean.csv')